Section 1 - Import Dependencies

In [47]:
#Import Libraries & Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import os
import gmaps
import scipy.stats as sts
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
from config import api_key_ggle

%matplotlib inline

Import files

In [48]:
#Load CSV containing from weatherpy out put
path = os.path.join('..','WeatherPy','Weatherhome.csv')
data_df = pd.read_csv(path)
data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ushuaia,-54.8000,-68.3000,41.00,87,20,6.91,AR,1613266121
1,Bluff,-46.6000,168.3333,56.59,84,100,5.50,NZ,1613266116
2,Rikitea,-23.1203,-134.9692,80.17,73,97,10.45,PF,1613266201
3,Punta Arenas,-53.1500,-70.9167,44.60,70,40,5.75,CL,1613265907
4,Vaini,-21.2000,-175.2000,86.00,74,20,10.36,TO,1613266201


In [49]:
# configure gmaps with API Key
gmaps.configure(api_key=api_key_ggle) 

In [50]:
#store 'Lat' and 'Lng' into locations
locations = data_df[['Lat', 'Lng']].astype(float)
#drop any NaN values in Humidity column
#hum = data_df.dropna()
humidity = data_df["Humidity"].astype(float)
#humidity = hum['Humidity'].astype(float)
#data_df = data_df.dropna()
humidity

0       87.0
1       84.0
2       73.0
3       70.0
4       74.0
       ...  
546     69.0
547    100.0
548     85.0
549     92.0
550     90.0
Name: Humidity, Length: 551, dtype: float64

In [53]:
fig = gmaps.figure(center=(0,0), zoom_level=2)
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 3,)
fig.add_layer(heatmap_layer)
fig
#map_type='HYBRID',

Figure(layout=FigureLayout(height='420px'))

In [54]:
locations

,Lat,Lng
0,-54.8000,-68.3000
1,-46.6000,168.3333
2,-23.1203,-134.9692
3,-53.1500,-70.9167
4,-21.2000,-175.2000
...,...,...
546,18.4167,-64.6167
547,36.4615,-77.6542
548,-24.5561,-54.0567
549,68.4140,15.9951


In [55]:
#Configure Gmaps and narrow down the DataFrame to find ideal weather conditions
#Drop any rows that don't contain all three conditions

condition_df = data_df.loc[(data_df['Max Temp']<80) & (data_df['Max Temp']>70)\
                & (data_df['Wind Speed']<10)\
                        &(data_df['Cloudiness']==0)]
condition_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,Saint-Leu,-21.1500,55.2833,73.99,60,0,5.23,RE,1613266204
35,Busselton,-33.6500,115.3333,73.00,61,0,3.00,AU,1613266209
79,Port Elizabeth,-33.9180,25.5701,73.40,94,0,6.91,ZA,1613266217
83,Bubaque,11.2833,-15.8333,77.92,63,0,8.90,GW,1613266217
90,Banikoara,11.2985,2.4386,72.90,18,0,4.63,BJ,1613266218
116,Ayorou,14.7308,0.9174,76.01,16,0,8.55,NE,1613266222
123,New Norfolk,-42.7826,147.0587,71.01,52,0,4.00,AU,1613266114
225,Calderitas,18.5500,-88.2500,77.25,81,0,6.20,MX,1613266236
229,Sémbé,1.6481,14.5806,74.41,77,0,3.31,CG,1613266237
283,Umluj,25.0213,37.2685,74.77,56,0,6.17,SA,1613266243


 Hotel Map

Store into variable named hotel_df.
Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.
Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.
Plot markers on top of the heatmap.

In [56]:
hotel_df=condition_df
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,Saint-Leu,-21.1500,55.2833,73.99,60,0,5.23,RE,1613266204
35,Busselton,-33.6500,115.3333,73.00,61,0,3.00,AU,1613266209
79,Port Elizabeth,-33.9180,25.5701,73.40,94,0,6.91,ZA,1613266217
83,Bubaque,11.2833,-15.8333,77.92,63,0,8.90,GW,1613266217
90,Banikoara,11.2985,2.4386,72.90,18,0,4.63,BJ,1613266218


In [57]:
hotel_df['Hotel Name']=""

In [69]:
# radius = "5000"
# unit = 'meters'
# #base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# params = {
#          "rankby":"radius",
#          "types":"hotel",
#          "keyword":"hotel",
#          "key":api_key_ggle}
# #use iterrows to iterate through pandas dataframe
# for index, row in hotel_df.iterrows():
    
#     base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#     lat =row['Lat']
#     long =row['Lng']
    
#     params['location']= f"{lat},{long}"
#     #print(f"printing results for Index {index}:hotel_type.")
    
#     response = requests.get(base_url, params=params).json()
    
    
#     try:
       
#         hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
#         print(f"closests Hotel is {response['results'][0]['name']}.")
#     except (IndexError):
#         print("Hotel not found... skipping.")

# hotel_df.head()   

In [68]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "key": api_key_ggle
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_address = requests.get(base_url, params=params)

    # convert to json
    hotel_address = hotel_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = hotel_address["results"][0]["vicinity"]
        print(f"Closest hotel is {hotel_address['results'][0]['name']}")
     
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head()       


Closest hotel is Saint-Leu, Réunion
Closest hotel is Busselton
Closest hotel is Port Elizabeth
Closest hotel is Bubaque
Closest hotel is Banikoara
Closest hotel is Ayorou
Closest hotel is New Norfolk
Closest hotel is Chetumal
Closest hotel is Sembe
Closest hotel is Umluj
Closest hotel is Inhambupe
Closest hotel is Formosa
Closest hotel is Vila Velha
Closest hotel is Yaguarón
Closest hotel is Mecca
Closest hotel is Tera
Closest hotel is Dabakala
Closest hotel is Natitingou


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
18,Saint-Leu,-21.1500,55.2833,73.99,60,0,5.23,RE,1613266204,"Saint-Leu, Réunion","Saint-Leu, Réunion"
35,Busselton,-33.6500,115.3333,73.00,61,0,3.00,AU,1613266209,Busselton,Busselton
79,Port Elizabeth,-33.9180,25.5701,73.40,94,0,6.91,ZA,1613266217,Port Elizabeth,Port Elizabeth
83,Bubaque,11.2833,-15.8333,77.92,63,0,8.90,GW,1613266217,Bubaque,Bubaque
90,Banikoara,11.2985,2.4386,72.90,18,0,4.63,BJ,1613266218,Banikoara,Banikoara


In [70]:
hotel_df=hotel_df.dropna()

In [71]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [72]:
# Add marker layer ontop of heat map


# Display figure

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
#checking number of cities limited
#condition_df.shape

(18, 9)

In [9]:
# #Concatenate and Convert the latitutude/longitude columns into a single column for aggregation
# condition_df['latLng']= condition_df['Lat'].astype(str) + "," + condition_df['Lng'].astype(str)
# condition_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,latLng
18,Saint-Leu,-21.1500,55.2833,73.99,60,0,5.23,RE,1613266204,"-21.15,55.2833"
35,Busselton,-33.6500,115.3333,73.00,61,0,3.00,AU,1613266209,"-33.65,115.3333"
79,Port Elizabeth,-33.9180,25.5701,73.40,94,0,6.91,ZA,1613266217,"-33.918,25.5701"
83,Bubaque,11.2833,-15.8333,77.92,63,0,8.90,GW,1613266217,"11.2833,-15.8333"
90,Banikoara,11.2985,2.4386,72.90,18,0,4.63,BJ,1613266218,"11.2985,2.4386"
116,Ayorou,14.7308,0.9174,76.01,16,0,8.55,NE,1613266222,"14.7308,0.9174"
123,New Norfolk,-42.7826,147.0587,71.01,52,0,4.00,AU,1613266114,"-42.7826,147.0587"
225,Calderitas,18.5500,-88.2500,77.25,81,0,6.20,MX,1613266236,"18.55,-88.25"
229,Sémbé,1.6481,14.5806,74.41,77,0,3.31,CG,1613266237,"1.6481,14.5806"
283,Umluj,25.0213,37.2685,74.77,56,0,6.17,SA,1613266243,"25.0213,37.2685"


In [42]:
# #Filter and Create data frame to hold required values needed to pick hotel
# #Data frame created to tie the coordinates for hotel search. nearby hotels within 5000 meters. 
# clean_cond_df =condition_df[['City', 'Humidity', 'Country', 'latLng']]          
# clean_cond_df



,City,Humidity,Country,latLng
18,Saint-Leu,60,RE,"-21.15,55.2833"
35,Busselton,61,AU,"-33.65,115.3333"
79,Port Elizabeth,94,ZA,"-33.918,25.5701"
83,Bubaque,63,GW,"11.2833,-15.8333"
90,Banikoara,18,BJ,"11.2985,2.4386"
116,Ayorou,16,NE,"14.7308,0.9174"
123,New Norfolk,52,AU,"-42.7826,147.0587"
225,Calderitas,81,MX,"18.55,-88.25"
229,Sémbé,77,CG,"1.6481,14.5806"
283,Umluj,56,SA,"25.0213,37.2685"


In [29]:
#set up 

In [30]:
# #Use Google places API to find first hotel for each city within 5000 meters
# radius = "10000"
# unit = 'meters'
# base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# params = {"location":"location",
#          "rankby":"radius",
#          "type":"hotel",
#          "key":"api_key_ggle",}

# #use iterrows to iterate through pandas dataframe
# for index, row in clean_cond.iterrows():
    
#     hotel_type =row['City']
    
#     params['keyword']= hotel_type
#     print(f"printing results for Index {index}:hotel_type.")
    
#     response = requests.get(base_url, params=params).json()
    
#     results =response["results"]
    
#     try:
#         print(f"closests{hotel_type}) Hotel is {results[0]['name']}.")
        
#     except (KeyError, IndexError):
#         print("Missing result ....skipping")

printing results for Index 18:hotel_type.
Missing result ....skipping
printing results for Index 35:hotel_type.
Missing result ....skipping
printing results for Index 79:hotel_type.
Missing result ....skipping
printing results for Index 83:hotel_type.
Missing result ....skipping
printing results for Index 90:hotel_type.
Missing result ....skipping
printing results for Index 116:hotel_type.
Missing result ....skipping
printing results for Index 123:hotel_type.
Missing result ....skipping
printing results for Index 225:hotel_type.
Missing result ....skipping
printing results for Index 229:hotel_type.
Missing result ....skipping
printing results for Index 283:hotel_type.
Missing result ....skipping
printing results for Index 320:hotel_type.
Missing result ....skipping
printing results for Index 351:hotel_type.
Missing result ....skipping
printing results for Index 411:hotel_type.
Missing result ....skipping
printing results for Index 440:hotel_type.
Missing result ....skipping
printing re

In [32]:
#Use Google places API to find first hotel for each city within 5000 meters

clean_cond_df = condition [["City....
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
#radius = "5000" # 1 mile proximity converted to meters

    #location = 

params = {
"radius": radius,
"query": ["hotel"],
"type": ['Name','City','Country'],
#"maxprice": max_price,
"key":api_key_ggle,
}

for i, row in clean_cond.iterrows():
    
    params["location"] = row[0]
    response = requests.get(base_url, params=params).json()
    try:
        clean_cond.loc[i,'hotel']= pprint(response) 
        #print(len(response['results']))
        #print(row[0])
    
    except:
        clean_cond.loc[i,'hotel']= np.nan

# = len(response['results'])
clean_cond
 #response.keys(). 
#print(response)

{'html_attributions': [],
 'next_page_token': 'ATtYBwJtUwWqzQhLhc3FxlUm1VALPwkd2Io-BxCMnozzI3Z7L-AlnQmA-aFck_TLQzJxJkxzU8iSZgsqW7JeO7qvqXXp2EG2GacaHZrVwgc090Lsqg_DGijoa1feGoPM8Q3j1fVYAT_989Ekioq2gmrNQWGWp13wFMSafe5BGhSMCCcPiRlVg89dSJaKrpp-GFWn5jxNPezEZyZRiZ2aTCvin-TQVCfvuU7I6o3Wl3AKNfxCQ_ZUd6awFC4r3QtevLVFzIPXZrsY7SAzpNyHN2bqPPH-udKMwD_Wn4lFkgW8_I-etnzEPi1gp0LU_ZUK3S-wl81qA9FxtUXfNMFm-fJ_xblettRVnhPtVAPziOmRAAVtvhG_0-TqrF2lFOZLhAnVlp_8qZGfe2NN',
 'results': [{'business_status': 'OPERATIONAL',
              'formatted_address': '976 W Pointe Ct, Lithia Springs, GA 30122, '
                                   'United States',
              'geometry': {'location': {'lat': 33.774702, 'lng': -84.606822},
                           'viewport': {'northeast': {'lat': 33.77605382989272,
                                                      'lng': -84.60536842010727},
                                        'southwest': {'lat': 33.77335417010727,
                                                 

{'html_attributions': [],
 'next_page_token': 'ATtYBwKgJnKpFeZVzj86YKJP7PxmGlC8wwnJ9KzB7glw_Ov4WrFiUzERPwtYdG4tznNZrVWuK1M3NrENC3jNNGsNgusI2SZ-Oe-gjWsGmKiOpoFFWfh1l8JSsH_ujis34Q2agLLgIu-nBhj4_gcbsE4Tw1SJ-u0KmGpA76ihqN0PUEnO3NTLxYJicHyOUeHs_MLMPol7H9fAKsaulbzkyjOxMCoj3U7kPd2Q4eoeFezH3Tzzknovox_YdyyeuA8WGai_Z9bYnxNjuG0PG3Om8wmdgXPQPGJMHV-SBfBmcV9U7fvhwQilgez4Oks9oNurIFZhZVqCZjhUgDnBg8wc8CecolsiS_8JgfmpoAKCzDMiTrwWgkEkVN8pwtGNYdA3Iy-NX3908phy0luc',
 'results': [{'business_status': 'OPERATIONAL',
              'formatted_address': '976 W Pointe Ct, Lithia Springs, GA 30122, '
                                   'United States',
              'geometry': {'location': {'lat': 33.774702, 'lng': -84.606822},
                           'viewport': {'northeast': {'lat': 33.77605382989272,
                                                      'lng': -84.60536842010727},
                                        'southwest': {'lat': 33.77335417010727,
                                                 

{'html_attributions': [],
 'next_page_token': 'ATtYBwJAxBMRTpmZBVpSKo63LAFA4fuLbsh34Z_NXJm9T3Z7H3m3U05lmHU4bEBjQktZMGS1tz91-0qmsaEwWEqZa1gMvmcJNF7puKIBAY8PeGdewbW4qPxMPYn1PuKzdhgCEadxlWjGocjU49-1idwx36bru8C9jDbhNO-z46na5LU8Jn1WmcoN9usLrUFldqzwdfQDNQaPWndz8SfTwbMq3gdml8ez3MNXjCz2_kgcQ0a47jAQi1i43QXQkW-BDQ0x9P5RLvz7xlctKlODOwGZ6ILvk0iwUbyeV3YLD7r9tRXuS31PvIPuPtQPhdrgL_1_zmm8PorjsCEg47Ra4smd2rJsrvY1tNDoBdw7IrA1nAwV1hn7Zn-cFD5xkTTXwslh-ndxEbwpjfTb',
 'results': [{'business_status': 'OPERATIONAL',
              'formatted_address': '976 W Pointe Ct, Lithia Springs, GA 30122, '
                                   'United States',
              'geometry': {'location': {'lat': 33.774702, 'lng': -84.606822},
                           'viewport': {'northeast': {'lat': 33.77605382989272,
                                                      'lng': -84.60536842010727},
                                        'southwest': {'lat': 33.77335417010727,
                                                 

{'html_attributions': [],
 'next_page_token': 'ATtYBwJmnCyOBcaIzBI1NaHelmnluTsLt6eV8JvGUoFbP4tSxpT9NVfREuuYQGh0AZ5s0Hg83f_xH7gkliBSE1tHmUqw8GcuyFk4n8pu43q0x2rFOaEDDvNl3oiElBd2pTyWTPfbIU_4Zr90uesPsjgluFkEz-vFAGAECUiG24YZGIJVuT6Ud5s7luOOO5BselL55RDvhZb2d-oABEfliv_VwJ621LOR3jWPyu2W44Bl3vwSMiFq-HwQmH39UH2C608GG06SgxNqglB8Q9G6xEnlBmFBKT51jCbyWadQBMHTU9L1L50ewQQP_q2UmhyKF9Rl9pZqaIpj29ROWCu_N3DamahvUHBW72GDzfptpqMdhQ6m1LUGhTOTYyr018gaIxiGa8MeJXd-S6uG',
 'results': [{'business_status': 'OPERATIONAL',
              'formatted_address': '976 W Pointe Ct, Lithia Springs, GA 30122, '
                                   'United States',
              'geometry': {'location': {'lat': 33.774702, 'lng': -84.606822},
                           'viewport': {'northeast': {'lat': 33.77605382989272,
                                                      'lng': -84.60536842010727},
                                        'southwest': {'lat': 33.77335417010727,
                                                 

{'html_attributions': [],
 'next_page_token': 'ATtYBwIDjRDUxxIYEniH6CXiL84rMXRpOd2nGX6vpLAIgX1LyI8kLgnCwRISYs8I1mEJJNqCZZ4USGB4bHmaI9hUZiUdGnPWctcUtnK_OlODfuSnQ9KV44GJgOaeODgL3fOGGSkO3pJpHG7OcIU2NM2OBluEsOoR4HmyVN7BhM-drWrOU0XnuxWnFOilPOLHfDfeonW4K-lM3UhcTjipH7HntMdhxzciyDmC-6i6zWdCZO2Btl8dZgBuAX8qco41-chpGCF3FL6eP8iSuH_GjqzUypbyWYtv0j3g4DKD_gf3XHCQseLmHksuaWpU7FZscmb6use3_ZUlWidLpNSqj0cuQgYGHRZ13nFlpcYkN0lrh7Io4pnB0AgywnDNAPsoTWRQjU82erZ1F3bW',
 'results': [{'business_status': 'OPERATIONAL',
              'formatted_address': '976 W Pointe Ct, Lithia Springs, GA 30122, '
                                   'United States',
              'geometry': {'location': {'lat': 33.774702, 'lng': -84.606822},
                           'viewport': {'northeast': {'lat': 33.77605382989272,
                                                      'lng': -84.60536842010727},
                                        'southwest': {'lat': 33.77335417010727,
                                                 

...

{'html_attributions': [],
 'next_page_token': 'ATtYBwIwQXeFUWSc4UcMJ--iid6U83EroaTD73wxsE7C_ywJo7SIj34tIlyw0EuX_T-hquj9FJOOX74lBuDuJNF53Y8dGw4IpVlbctoLQHKthLJcWRbahOZ92tF-Kz73AkOZX4w-PMW05hm0743n8iJka6D3cLBIUqO1Ti643eQhLZGcIPrecYLE3nTMpMQCo8-6vdrIn6JaIsj7PM52dgmaO0of1Fu2FfRxZOQ-4onCy92eWZoGq6deMfEgWBHhiWIkUTS74-rBJIolUanMNa1qANedByhTwnD6j3FrjZaiNUkh-_6FZtEVb0NHDWPVdYLOu8NtMTQiTX1meZpR9jr9sMU5hmx5pvgY-t58mr6dSIQIkULkVkz7pax7BMyFD1-dgShIr4Z7TsCs',
 'results': [{'business_status': 'OPERATIONAL',
              'formatted_address': '976 W Pointe Ct, Lithia Springs, GA 30122, '
                                   'United States',
              'geometry': {'location': {'lat': 33.774702, 'lng': -84.606822},
                           'viewport': {'northeast': {'lat': 33.77605382989272,
                                                      'lng': -84.60536842010727},
                                        'southwest': {'lat': 33.77335417010727,
                                                 

{'html_attributions': [],
 'next_page_token': 'ATtYBwK5umT7WXOLlUxK8Wlu6979MTLYvHuoYWiOBaW6KsQ7bJ77zOcEPOLpHn5_lAqlMrRvV_yHNVjvOHeIXJKBmrzr0K4-JndLErYfGNW2Pb-7A7CTHeinUdwaHnkZxthl6nzHnKq7aL3wk7fYRRooYjuvvYcbNPPehhiWi-9O4Ama7HDAMCWlkUQz7MauDFMGp5SMOh_7R_gRNrFuxZgsbiOq-peUC1-EbErK2LyZCAT0Ip4XvqCvkaOWlnlkRJZmmLZze5bQSDljTEcq5hsD_DcRFfjjyOwYyQT9HeoRs5KL3h0xeL9483iORLntHeahpvxaGLPWRGIbNjljEqY2otmpzDsLW80Jpx2gRz-yK5ubD6tuvTco9jAeXldDoIXpSYbRkk0Z476j',
 'results': [{'business_status': 'OPERATIONAL',
              'formatted_address': '976 W Pointe Ct, Lithia Springs, GA 30122, '
                                   'United States',
              'geometry': {'location': {'lat': 33.774702, 'lng': -84.606822},
                           'viewport': {'northeast': {'lat': 33.77605382989272,
                                                      'lng': -84.60536842010727},
                                        'southwest': {'lat': 33.77335417010727,
                                                 

{'html_attributions': [],
 'next_page_token': 'ATtYBwKhLPdmuLVodmypoc7SQ-2JcKhbPVBTNzguF1L6Cc-Tap4GPjS2HaLUU0OWA3Vl2Z6_ByMQGqDw2ujDbHKeQsNgWCNW9TMUF2gCYJseCDrtSa0PIyXlvzWNCKDL1hgpLs3R2GJQ-QNt2kyZ2yNEiI0psl5oYLngl4yNMekUQj7YALxN6-n7l0VBiK2qB1ctOOuA880AAVT3A0NzVjRJu-Wpr7tvKrdJvDGAYrIboOK6CBDNeI10hYQqO5LXTObM9Xf9NkTo6ukhVFpW92Ji8D9fEJ_xuq6EmeZiHcxzrTx4MR8_PDjNJg2k_SHebi2Npa9bZC3is3h9_q6hvH5P1LnPAGmo-0AeW__sateLzTmgaZ_rLgSSMO2WDpsYXyOqhaIe2WKjkDjg',
 'results': [{'business_status': 'OPERATIONAL',
              'formatted_address': '976 W Pointe Ct, Lithia Springs, GA 30122, '
                                   'United States',
              'geometry': {'location': {'lat': 33.774702, 'lng': -84.606822},
                           'viewport': {'northeast': {'lat': 33.77605382989272,
                                                      'lng': -84.60536842010727},
                                        'southwest': {'lat': 33.77335417010727,
                                                 

{'html_attributions': [],
 'next_page_token': 'ATtYBwIDh3lMHP8shyrvx1JdteV_Akx42NKg_KayqAt2Hl64pCNf0aR4XAo3d1KHVnxCID00H9lAY0FIAyCBh1BXgofiXm4vUbqBl2f3XwlSeW7fE32kJr8v1j-wqDrxBw7BgWnUx6_bvUXLoQSnf39MSqfsiyJlkD4piU7LB30OuahIszQDfhrmFsnCwIVFoFxq-ir_QYEmIZtMwbRWipOV6aYQqQx1YEuBLho_QIzpYzam6OPT1Jz3ZhzgagPJquTZDnhI35Zeblt7DLwESHtASJ3926i2ueTHpYLB0BYtcQC0tUIEGWCJJ0bEdDPj3cUaK76heW2kG5FlOvV5ktmhgFTsT8sEcSCIYihd5QNzYEcTpT4mEH-NdiL38GuVkzhe1_a8tFmRWYD7',
 'results': [{'business_status': 'OPERATIONAL',
              'formatted_address': '976 W Pointe Ct, Lithia Springs, GA 30122, '
                                   'United States',
              'geometry': {'location': {'lat': 33.774702, 'lng': -84.606822},
                           'viewport': {'northeast': {'lat': 33.77605382989272,
                                                      'lng': -84.60536842010727},
                                        'southwest': {'lat': 33.77335417010727,
                                                 

,City,Humidity,Country,latLng,hotel
18,Saint-Leu,60,RE,"-21.15,55.2833",NaN
35,Busselton,61,AU,"-33.65,115.3333",NaN
79,Port Elizabeth,94,ZA,"-33.918,25.5701",NaN
83,Bubaque,63,GW,"11.2833,-15.8333",NaN
90,Banikoara,18,BJ,"11.2985,2.4386",NaN
116,Ayorou,16,NE,"14.7308,0.9174",NaN
123,New Norfolk,52,AU,"-42.7826,147.0587",NaN
225,Calderitas,81,MX,"18.55,-88.25",NaN
229,Sémbé,77,CG,"1.6481,14.5806",NaN
283,Umluj,56,SA,"25.0213,37.2685",NaN


In [ ]:
#Define Figure Layout
figure_layout = {
    
    'width':'1000px',
    'height':'600px',
    'border':'1px solid black',
    'padding':'1px',
    'margin':'0 auto 0 auto'
    }

#Define Symbol Layers
f = gmaps.figure(layout=figure_layout, center=(33.7490, -84.386330),zoom_level=13, map_type='TERRAIN')
f.add_layer(gmaps.symbol_layer(coordg1, fill_color=(244, 27, 63, 0.8), stroke_color=(244, 27, 63, 0.8), scale=2))
f.add_layer(gmaps.symbol_layer(coordg2, fill_color=(38, 156, 98, 0.8), stroke_color=(38, 156, 98, 0.8), scale=2)

In [37]:
 
 # 6d. Set up the base URL for the Google Directions API to get JSON data.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
hotels = requests.get(base_url, params=params).json()
    
for index, row in clean_cond.iterrows():
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [43]:
clean_cond_df.head(10)


,City,Humidity,Country,latLng
18,Saint-Leu,60,RE,"-21.15,55.2833"
35,Busselton,61,AU,"-33.65,115.3333"
79,Port Elizabeth,94,ZA,"-33.918,25.5701"
83,Bubaque,63,GW,"11.2833,-15.8333"
90,Banikoara,18,BJ,"11.2985,2.4386"
116,Ayorou,16,NE,"14.7308,0.9174"
123,New Norfolk,52,AU,"-42.7826,147.0587"
225,Calderitas,81,MX,"18.55,-88.25"
229,Sémbé,77,CG,"1.6481,14.5806"
283,Umluj,56,SA,"25.0213,37.2685"


In [39]:
hotel_df.head(10)

NameError: name 'hotel_df' is not defined

In [46]:
#clean_cond_df = condition [["City....
params = {
    "radius": 5000,
    "type": "lodging",
    "key": api_key_ggle
}

# 6b. Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

NameError: name 'hotel_df' is not defined

In [40]:
clean_cond_df.head(10)

NameError: name 'clean_cond_df' is not defined

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
radius = "1609.34" # 1 mile proximity converted to meters
max_price = "2" # maximum price set to level 2

    #location = 

params = {
"radius": radius,
"query": ["ice cream shop"],
"type": ['cafe','restaurant','drugstore'],
"maxprice": max_price,
"key": gkey,
}

for i, row in cityLL.iterrows():
    
    params["location"] = row[0]
    response = requests.get(base_url, params=params).json()
    try:
        cityLL.loc[i,'store count']= len(response['results'])
        #print(len(response['results']))
        #print(row[0])
    except:
        cityLL.loc[i,'store count']= np.nan